In [6]:
# Рекомендуемое окружение: Python 3.8+
!pip install torch torchvision transformers pillow numpy ultralytics tqdm
# Если планируете BLIP VQA из репозитория salesforce/BLIP:
!pip install git+https://github.com/salesforce/BLIP.git@main
# (опционально) если хотите yolov8:
!pip install ultralytics


pip is disabled in bash. Please use %pip magic.


Exception: Process exited with code 1

In [7]:
#!/usr/bin/env python3
# multimodel_llava_pipeline.py
# Объединяет: rotation correction (resnet50), object detection (DETR / optional YOLO),
# environment crop (нижняя часть под машиной), BLIP captioning + optional BLIP VQA (если есть).

import os
import sys
import json
from pathlib import Path
from typing import Dict, Any, List, Optional

import numpy as np
from PIL import Image
from tqdm import tqdm

import torch

# Transformers / BLIP
from transformers import BlipProcessor, BlipForConditionalGeneration, DetrImageProcessor, DetrForObjectDetection

# Optional modules
try:
    from ultralytics import YOLO
    _HAS_YOLO = True
except Exception:
    YOLO = None
    _HAS_YOLO = False

# Optional BLIP VQA from local BLIP repo (if installed via pip from github)
try:
    # from models.blip_vqa import blip_vqa  # this will work if BLIP repo is in python path
    from models.blip_vqa import blip_vqa  # type: ignore
    _HAS_BLIP_VQA = True
except Exception:
    blip_vqa = None
    _HAS_BLIP_VQA = False

# Optional rotation model (resnet50)
try:
    import torch.nn as nn
    from torchvision import models, transforms
    _HAS_TORCHVISION = True
except Exception:
    _HAS_TORCHVISION = False

# ========== Настройки (отредактируйте пути) ==========
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

IMAGE_FOLDER = "/home/jupyter/project/Ilya/dataset_images"  # <- поменяйте под себя
OUTPUT_JSON = "/home/jupyter/project/Ilya/multimodel_results.json"

# Если локально есть модель поворота (resnet50 trained with 4 classes 0/90/180/270)
ROTATION_MODEL_PATH = "../MODELS/resnet50_rotation_car_99.76.pth"  # если нет — пропустит шаг поворота

# Если у вас есть yolov8 модель (опционально) — укажите путь
YOLO_MODEL_PATH = "../MODELS/yolov8x-oiv7.pt"  # optional

# DETR threshold
DETR_THRESHOLD = 0.5

# Какие COCO классы считаем "машиной"
VEHICLE_CLASSES = {"car", "truck", "bus", "motorcycle", "train", "van", "taxi"}

# Вопрос для VQA (пример)
QUESTION = "Describe only the surroundings near the car, such as buildings, trees, street lights, signs, or pavement. Do not mention the car, vehicle, sky, or clouds."

# ======================================================

# ========== Загрузка моделей трансформеров ==========
print("Loading BLIP captioning...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(DEVICE)

print("Loading DETR (object detection)...")
detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50").to(DEVICE)

# Попытка загрузить YOLO, если есть и файл указан
yolo_model = None
if _HAS_YOLO and YOLO is not None and os.path.exists(YOLO_MODEL_PATH):
    try:
        print("Loading YOLO model (optional)...")
        yolo_model = YOLO(YOLO_MODEL_PATH)
    except Exception as e:
        print("Не удалось загрузить YOLO:", e)
        yolo_model = None
else:
    if _HAS_YOLO:
        print("YOLO установлен, но путь к модели не найден или модель не указана:", YOLO_MODEL_PATH)
    else:
        print("ultralytics (YOLO) не установлен — пропускаем.")

# Попытка загрузить BLIP VQA (опционально)
blip_vqa_model = None
if _HAS_BLIP_VQA and blip_vqa is not None:
    try:
        print("Loading BLIP VQA (optional) from BLIP repo...")
        # пример download URL - если вы хотите использовать готовую pretrained weight, укажите его как в вашем примере
        # model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'
        # model = blip_vqa(pretrained=model_url, image_size=480, vit='base', med_config=...)
        # В этом обобщённом скрипте попытаемся создать модель без weights (пользователь обязан настроить)
        blip_vqa_model = None
        # Если вы хотите использовать BLIP VQA — загрузите вручную модель и замените blip_vqa_model = model
        print("BLIP VQA доступен, но автоматическая загрузка весов не настроена. Если хотите — укажите путь/URL и раскомментируйте загрузку.")
    except Exception as e:
        print("BLIP VQA не удалось инициализировать:", e)
        blip_vqa_model = None
else:
    print("BLIP VQA недоступен (BLIP repo не найден). VQA будет выполнен через BLIP captioning fallback.")

# ========== Rotation model (опционально) ==========
rotation_model = None
rotation_transform = None
angle_values = {0: 0, 1: 90, 2: 180, 3: 270}

if _HAS_TORCHVISION and os.path.exists(ROTATION_MODEL_PATH):
    try:
        print("Loading rotation model (resnet50)...")
        rotation_model = models.resnet50(weights=None)
        num_features = rotation_model.fc.in_features
        rotation_model.fc = nn.Linear(num_features, 4)
        rotation_model.load_state_dict(torch.load(ROTATION_MODEL_PATH, map_location=DEVICE))
        rotation_model = rotation_model.to(DEVICE)
        rotation_model.eval()

        rotation_transform = transforms = __import__("torchvision.transforms", fromlist=["transforms"]).transforms.Compose([
            __import__("torchvision.transforms", fromlist=["transforms"]).transforms.Resize((224, 224)),
            __import__("torchvision.transforms", fromlist=["transforms"]).transforms.ToTensor(),
            __import__("torchvision.transforms", fromlist=["transforms"]).transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                                           std=[0.229, 0.224, 0.225])
        ])
    except Exception as e:
        print("Не удалось загрузить rotation model:", e)
        rotation_model = None
else:
    print("Rotation model не найден или torchvision отсутствует — шаг поворота пропускается.")

# ========== Функции обработки ==========
def correct_rotation_if_available(pil_image: Image.Image) -> Image.Image:
    """Если модель поворота доступна, предсказывает класс и разворачивает изображение."""
    if rotation_model is None or rotation_transform is None:
        return pil_image
    try:
        input_tensor = rotation_transform(pil_image).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            out = rotation_model(input_tensor)
            pred = int(torch.argmax(out, dim=1).item())
        angle = angle_values.get(pred, 0)
        if angle != 0:
            return pil_image.rotate(-angle, expand=True)
        return pil_image
    except Exception as e:
        print("Rotation correction failed:", e)
        return pil_image

def detect_vehicles_detr(pil_image: Image.Image, threshold: float = DETR_THRESHOLD) -> List[np.ndarray]:
    """Возвращает список bbox'ов (xmin, ymin, xmax, ymax) обнаруженных как vehicles по DETR."""
    try:
        inputs = detr_processor(images=pil_image, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            outputs = detr_model(**inputs)
        target_sizes = torch.tensor([pil_image.size[::-1]]).to(DEVICE)
        results = detr_processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=threshold)[0]
        boxes = []
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            class_name = detr_model.config.id2label[int(label.item())]
            if class_name.lower() in VEHICLE_CLASSES:
                boxes.append(box.cpu().numpy())
        return boxes
    except Exception as e:
        print("DETR detection failed:", e)
        return []

def detect_vehicles_yolo(pil_image: Image.Image) -> List[np.ndarray]:
    """Если есть yolov8, возвращает list of boxes (xmin, ymin, xmax, ymax) для vehicle-like classes."""
    if yolo_model is None:
        return []
    try:
        results = yolo_model(pil_image, verbose=False)
        r = results[0]
        boxes = []
        for box in r.boxes:
            cls_id = int(box.cls)
            cls_name = r.names[cls_id].lower()
            if cls_name in VEHICLE_CLASSES or "car" in cls_name or "vehicle" in cls_name:
                xyxy = box.xyxy.cpu().numpy().tolist()  # [xmin, ymin, xmax, ymax]
                boxes.append(np.array(xyxy))
        return boxes
    except Exception as e:
        print("YOLO detection failed:", e)
        return []

def crop_to_environment_from_boxes(pil_image: Image.Image, vehicle_boxes: List[np.ndarray]) -> Image.Image:
    """Обрезает изображение, оставляя область под самым нижним автомобилем (или нижнюю половину, если нет box)."""
    arr = np.array(pil_image)
    h, w = arr.shape[:2]
    if not vehicle_boxes:
        return Image.fromarray(arr[h//2:, :])

    # Найдём самый нижний y_max
    ymaxs = [box[3] for box in vehicle_boxes]
    max_ymax = max(ymaxs)
    crop_y1 = int(max_ymax)
    if crop_y1 >= h:
        crop_y1 = h // 2
    cropped = arr[crop_y1:, :]
    if cropped.size == 0:
        cropped = arr[h//2:, :]
    return Image.fromarray(cropped)

def generate_captions(env_image: Image.Image) -> Dict[str, str]:
    """Генерирует три варианта подписи через BLIP."""
    try:
        inputs = blip_processor(env_image, return_tensors="pt").to(DEVICE)

        with torch.no_grad():
            outputs = blip_model.generate(**inputs, max_length=50, num_beams=5, early_stopping=True)
            base = blip_processor.decode(outputs[0], skip_special_tokens=True)

            detailed_outputs = blip_model.generate(**inputs, max_length=100, num_beams=7, length_penalty=2.0, early_stopping=True)
            detailed = blip_processor.decode(detailed_outputs[0], skip_special_tokens=True)

            alt_outputs = blip_model.generate(**inputs, max_length=60, do_sample=True, temperature=0.9, top_p=0.9)
            alt = blip_processor.decode(alt_outputs[0], skip_special_tokens=True)

        return {"base": base, "detailed": detailed, "alternative": alt}
    except Exception as e:
        print("BLIP caption failed:", e)
        return {"base": "Ошибка", "detailed": "Ошибка", "alternative": "Ошибка"}

def answer_vqa(env_image: Image.Image, question: str) -> str:
    """Пытается ответить на вопрос. Если BLIP VQA доступен — использует её, иначе выполняет fallback (BLIP caption + prompt)."""
    # 1) Если BLIP VQA модель доступна (пользователь должен настроить загрузку) — вызвать её.
    if blip_vqa_model is not None:
        try:
            # Placeholder: конкретный API зависит от реализации blip_vqa interface
            # Пример: answer = blip_vqa_model(image_tensor, [question], train=False, inference='generate')
            # return answer[0]
            return "VQA через BLIP VQA (локально) — ответ не настроен в примере."
        except Exception as e:
            print("BLIP VQA failed:", e)

    # 2) Fallback: сконкатенировать question с prompt и прогнать BLIP captioning (генеративно)
    try:
        # Для более качественного ответа формируем подсказку: добавим instruction + question
        instruction = f"Answer the question about the environment near the car. Question: {question}"
        # BLIP captioning expects just an image; мы попробуем передать instruction в decoder как prompt (форсируем через input_ids)
        # Простая стратегия: сгенерировать несколько caption'ов и вернуть наиболее подходящий (detailed)
        caps = generate_captions(env_image)
        # Простейший "fake VQA": вернуть detailed caption — пользователь может захотеть более точную реализацию
        return caps["detailed"]
    except Exception as e:
        print("VQA fallback failed:", e)
        return "Ошибка при попытке ответить на VQA."

# ========== Main loop ==========
def process_folder(image_folder: str, output_json: str):
    image_folder = Path(image_folder)
    assert image_folder.exists(), f"Папка не найдена: {image_folder}"

    image_extensions = {'.png', '.jpg', '.jpeg', '.bmp'}
    files = [p for p in sorted(image_folder.iterdir()) if p.suffix.lower() in image_extensions]
    print(f"Найдено изображений: {len(files)}")

    results: Dict[str, Any] = {}

    for i, p in enumerate(tqdm(files, desc="Processing")):
        fname = p.name
        try:
            pil = Image.open(p).convert("RGB")
        except Exception as e:
            print(f"Cannot open {p}: {e}")
            continue

        # 1) rotation
        pil_rot = correct_rotation_if_available(pil)

        # 2) detect vehicles (yolo optional, then DETR fallback)
        boxes = []
        if yolo_model is not None:
            boxes = detect_vehicles_yolo(pil_rot)
        if not boxes:
            boxes = detect_vehicles_detr(pil_rot)

        # 3) crop environment
        env_img = crop_to_environment_from_boxes(pil_rot, boxes)

        # 4) captions
        captions = generate_captions(env_img)

        # 5) vqa
        vqa_answer = answer_vqa(env_img, QUESTION)

        # 6) aggregate
        results[fname] = {
            "captions": captions,
            "vqa_answer": vqa_answer,
            "detected_vehicle_boxes": [box.tolist() for box in boxes],
            "rotated": pil_rot.size if pil_rot is not None else None,
            "env_crop_size": env_img.size
        }

        # краткий прогресс в stdout
        print(f"[{i+1}/{len(files)}] {fname}")
        print("  Base:", captions["base"])
        print("  VQA:", vqa_answer)
        print()

    # сохранить json
    os.makedirs(os.path.dirname(output_json), exist_ok=True)
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print("Done. Results saved to:", output_json)
    return results

if __name__ == "__main__":
    process_folder(IMAGE_FOLDER, OUTPUT_JSON)


Device: cuda
Loading BLIP captioning...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8c545b3d-7987-4e8c-9fdc-1e3dc6616f68)')' thrown while requesting HEAD https://huggingface.co/Salesforce/blip-image-captioning-base/resolve/main/preprocessor_config.json
Retrying in 1s [Retry 1/5].


Loading DETR (object detection)...
YOLO установлен, но путь к модели не найден или модель не указана: ../MODELS/yolov8x-oiv7.pt
BLIP VQA недоступен (BLIP repo не найден). VQA будет выполнен через BLIP captioning fallback.
Loading rotation model (resnet50)...
Найдено изображений: 179


Processing:   1%|          | 1/179 [00:01<04:18,  1.45s/it]

[1/179] img_00000.jpeg
  Base: a white van parked on the side of a road
  VQA: a white van parked on the side of a road



...

[177/179] img_00197.jpeg
  Base: a white car parked in a parking lot
  VQA: a white car is parked in a parking lot



Processing:  99%|█████████▉| 178/179 [04:14<00:01,  1.37s/it]

[178/179] img_00198.jpeg
  Base: a black and white photo of a man in a suit
  VQA: a black and white photo of a man in a suit



Processing: 100%|██████████| 179/179 [04:15<00:00,  1.43s/it]

[179/179] img_00199.jpeg
  Base: a man riding a bike down a dirt road
  VQA: a man riding a bike down a dirt road

Done. Results saved to: /home/jupyter/project/Ilya/multimodel_results.json


In [42]:
%pip install timm torchvision

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 73.6 MB/s  0:00:016m0:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.5.0
    Uninstalling triton-3.5.0:
      Successfully uninstalled triton-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.2+cu118 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.2+cu118 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 2.1.2+cu118 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [43]:
%pip install --upgrade transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [41]:
%python -c "import transformers; print(transformers.__version__)"

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [44]:
#!/usr/bin/env python3
# llava_integration.py
import os
import json
from pathlib import Path
from PIL import Image
import torch

# --- Настройки ---
IMAGES_DIR = Path("/home/jupyter/project/Ilya/dataset_images")
OUTPUT_DIR = Path("/home/jupyter/project/Ilya/llava_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Путь к ноутбуку/модулю с run_pipeline
NOTEBOOK_PATH = Path("/home/jupyter/project/github/car-scene-captioning/main.ipynb")
# или если вы скопировали/создали run_pipeline.py рядом - укажите его:
RUNTIME_MODULE_PATH = Path("/home/jupyter/project/github/car-scene-captioning/run_pipeline.py")

# Модель LLaVA — пример идентификатора; поменяйте на нужный
LLAVA_MODEL_ID = "llava-hf/llava-1.5-7b-hf"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- helper: получить run_pipeline из .py или .ipynb ---
def load_run_pipeline():
    # 1) Попробовать импортировать из .py модуля если есть
    if RUNTIME_MODULE_PATH.exists():
        import importlib.util
        spec = importlib.util.spec_from_file_location("run_pipeline_module", str(RUNTIME_MODULE_PATH))
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        if hasattr(module, "run_pipeline"):
            return module.run_pipeline

    # 2) Если есть ноутбук, собрать и выполнить код его кодовых ячеек
    if NOTEBOOK_PATH.exists():
        import nbformat
        nb = nbformat.read(str(NOTEBOOK_PATH), as_version=4)
        code_cells = [c.source for c in nb.cells if c.cell_type == "code"]
        code = "\n\n".join(code_cells)
        ns = {}
        # Выполним код ноутбука в изолированном namespace
        exec(compile(code, str(NOTEBOOK_PATH), "exec"), ns)
        if "run_pipeline" in ns:
            return ns["run_pipeline"]

    raise RuntimeError("Не удалось найти функцию run_pipeline ни в run_pipeline.py, ни в main.ipynb. Скопируйте/экспортируйте её в доступный модуль.")

# --- Загрузка LLaVA (inference) ---
def load_llava(model_id=LLAVA_MODEL_ID, device=DEVICE):
    # попытка импортировать типичный API, используемый в примерах LLaVA
    try:
        # некоторые реализации предоставляют классы напрямую
        from llava.modeling import LlavaForConditionalGeneration
        from transformers import AutoProcessor
        model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)
        processor = AutoProcessor.from_pretrained(model_id)
    except Exception:
        # fallback — часто можно импортировать напрямую из transformers or llava-hf wrappers
        try:
            from transformers import AutoProcessor
            from llava import LlavaForConditionalGeneration  # возможные импорты в разных реализациях
            model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)
            processor = AutoProcessor.from_pretrained(model_id)
        except Exception:
            # более общий вариант — использовать AutoModel/AutoTokenizer (пользователь может поправить по своему стеку)
            from transformers import AutoProcessor, AutoModelForCausalLM
            processor = AutoProcessor.from_pretrained(model_id)
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)

    model.to(device)
    model.eval()
    return processor, model

# --- Формирование промпта: комбинируем результаты вашего pipeline и вопрос для LLaVA ---
def build_prompt(pipeline_result: list):
    # pipeline_result — список строк/словарей, которые возвращает ваш run_pipeline для одного изображения
    # Пример шаблона — вы можете адаптировать под задачу:
    header = "Информация, полученная preprocessing pipeline:\n"
    body_lines = []
    for item in pipeline_result:
        # если это словарь — представим его красиво
        if isinstance(item, dict):
            body_lines.append(json.dumps(item, ensure_ascii=False))
        else:
            body_lines.append(str(item))
    body = "\n".join(body_lines)
    # вопрос/задача для LLaVA
    question = (
        "\n\nЗадача: на основании изображения и информации выше опишите кратко автомобиль, "
        "окрестность и перечислите обнаруженные объекты. Выдайте ответ в двух частях:\n"
        "1) Краткая сводка (1-2 предложения)\n"
        "2) JSON с полями: objects (список), scene (строка), potential_hazards (строка, если есть)\n"
    )
    return header + body + question

# --- Основной run: перебор изображений, запуск pipeline, запуск llava и сохранение результата ---
def main():
    run_pipeline = load_run_pipeline()
    processor, model = load_llava()
    # Список картинок
    image_paths = sorted([p for p in IMAGES_DIR.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])

    batch_paths = [str(p) for p in image_paths]
    # Запускаем ваш pipeline (он сам инициализирует модели внутри себя)
    print("Запуск вашего run_pipeline для всех изображений...")
    pipeline_results = run_pipeline(batch_paths, source=True)  # ожидается list[list[...]]
    print("Pipeline завершён.")

    for idx, img_path in enumerate(image_paths):
        img = Image.open(img_path).convert("RGB")

        # pipeline_results[idx] — список результатов для конкретного изображения
        prompt = build_prompt(pipeline_results[idx])

        # Подготовка входа для LLaVA: processor соединит изображение и текст
        inputs = processor(images=img, text=prompt, return_tensors="pt")
        # Переносим на устройство
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # Генерация — параметры подберите по ресурсам
        with torch.no_grad():
            gen = model.generate(**inputs, max_new_tokens=256, do_sample=False, num_beams=4)

        # Декодирование — у processor/токенайзера может быть метод decode
        try:
            generated_text = processor.tokenizer.decode(gen[0], skip_special_tokens=True)
        except Exception:
            # fallback если processor не содержит tokenizer
            from transformers import AutoTokenizer
            tok = AutoTokenizer.from_pretrained(LLAVA_MODEL_ID)
            generated_text = tok.decode(gen[0], skip_special_tokens=True)

        # Сохраняем
        out = {
            "image": str(img_path),
            "pipeline": pipeline_results[idx],
            "llava_output": generated_text
        }
        out_path = OUTPUT_DIR / (img_path.stem + "_llava.json")
        with open(out_path, "w", encoding="utf-8") as f:
            json.dump(out, f, ensure_ascii=False, indent=2)

        print(f"Обработано {img_path.name} -> {out_path}")

if __name__ == "__main__":
    main()


init


RuntimeError: Failed to import transformers.models.mask2former.image_processing_mask2former because of the following error (look up to see its traceback):
No module named 'transformers.models.mask2former.image_processing_mask2former'

In [46]:
%pip -q install --upgrade "transformers>=4.35.0" "timm>=0.6.0" "torchvision>=0.15.0"

Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_parser.py", line 73, in _parse_requirement
    name_token = tokenizer.expect(
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_tokenizer.py", line 140, in expect
    raise self.raise_syntax_error(f"Expected {expected}")
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_tokenizer.py", line 165, in raise_syntax_error
    raise ParserSyntaxError(
ml_kernel._vendor.packaging._tokenizer.ParserSyntaxError: Expected package name at the start of dependency specifier
    "transformers>=4.35.0"
  

In [29]:
# обновить до последней стабильной
%pip install -U transformers

# (опционально) если хотите конкретную более новую версию:
%pip install -U "transformers>=4.39.0"


Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers━━━━━━━━━━━━ 0/2 [tokenizers]
    Found existing installation: transformers 4.30.02m0/2 [tokenizers]
    Uninstalling transformers-4.30.0:╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.30.0━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers] [transformers]
ERROR: pip's dependency resolver does n

Traceback (most recent call last):
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/requirements.py", line 35, in __init__
    parsed = _parse_requirement(requirement_string)
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_parser.py", line 64, in parse_requirement
    return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_parser.py", line 73, in _parse_requirement
    name_token = tokenizer.expect(
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_tokenizer.py", line 140, in expect
    raise self.raise_syntax_error(f"Expected {expected}")
  File "/kernel/lib/python3.10/site-packages/ml_kernel/_vendor/packaging/_tokenizer.py", line 165, in raise_syntax_error
    raise ParserSyntaxError(
ml_kernel._vendor.packaging._tokenizer.ParserSyntaxError: Expected package name at the start of dependency specifier
    "transformers>=4.39.0"
  

In [37]:
# вариант 1: через "!"
!python3 -m venv ~/venv_autocaption
!source ~/venv_autocaption/bin/activate && pip install --upgrade pip
!source ~/venv_autocaption/bin/activate && pip install torch torchvision pillow numpy


The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /home/jupyter/venv_autocaption/bin/python3



Exception: Process exited with code 1

In [30]:
python - <<'PY'
from transformers import Mask2FormerImageProcessor, Mask2FormerForUniversalSegmentation
print("Mask2Former импортируется ✓")
PY


SyntaxError: invalid syntax (763689901.py, line 1)